# Exercise 1

In [2]:
import copy

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Import CIFAR-10

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root="../CIFAR10/",
                                        train=True, 
                                        download=True, 
                                        transform=transform)
train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])
trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=4, 
                                          shuffle=True)
validationloader = torch.utils.data.DataLoader(validationset,
                                          batch_size=1,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='../CIFAR10/',
                                       train=False,
                                       download=True, 
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=1,
                                         shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


### CNN

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 84)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.tanh(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        x = self.fc2(x)

        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net().to(device)

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0001)

writer = SummaryWriter()



### Training

In [5]:
best_accuracy = 0
best_net = 0
for epoch in range(15):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(trainloader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(validationloader, 0):
        inputs, labels = data

        outputs = net(inputs.to(device))
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(net)
        print(" (new best)")

print('\nFinished Training')
writer.flush()

/Users/simon/Documents/D7047E-AdvancedDeepLearning/venv/lib/python3.8/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 1 [10000/10000] - Loss: 1.4720083475112915, accuracy:  0.4321
 (new best)
Epoch 2 [10000/10000] - Loss: 0.7810292243957524, accuracy:  0.4789
 (new best)
Epoch 3 [10000/10000] - Loss: 1.6004712581634521, accuracy:  0.5102
 (new best)
Epoch 4 [10000/10000] - Loss: 1.8340728282928467, accuracy:  0.5364
 (new best)
Epoch 5 [10000/10000] - Loss: 1.5625126361846924, accuracy:  0.5537
 (new best)
Epoch 6 [10000/10000] - Loss: 1.5329855680465698, accuracy:  0.5709
 (new best)
Epoch 7 [10000/10000] - Loss: 1.8760881423950195, accuracy:  0.5792
 (new best)
Epoch 8 [10000/10000] - Loss: 0.7753048539161682, accuracy:  0.6011
 (new best)
Epoch 9 [10000/10000] - Loss: 0.7043122053146362, accuracy:  0.6001
Epoch 10 [10000/10000] - Loss: 0.90804523229599673, accuracy:  0.6101
 (new best)
Epoch 11 [10000/10000] - Loss: 0.7240840792655945, accuracy:  0.6144
 (new best)
Epoch 12 [10000/10000] - Loss: 1.4367082118988037, accuracy:  0.6108
Epoch 13 [10000/10000] - Loss: 2.4656381607055664, accuracy:

### Test

In [6]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
    inputs, labels = data

    outputs = best_net(inputs.to(device))
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.6289
